# Algoritmos de región de confianza

### Usando método DogLeg

In [8]:
import numpy as np
from numpy import linalg as la 
from functools import partial

def Modify_hessian(x_k,H,beta=1e-3):
    B_k=H(x_k)  #Esto saca la aproximación del hessiano
    if min(B_k.diagonal())>0:
        tau=0
    else:
        tau=-min(B_k.diagonal())+beta
    Aux=B_k+tau*np.identity(B_k.shape[0])
    for i in range(1000):
        try:
            la.cholesky(Aux)
            B_k=Aux
            return B_k
            break
        except:
            tau=max(2*tau,beta)
            Aux=B_k+tau*np.identity(B_k.shape[0])
    
def resolver_taw_para_dogleg(pu, pb, delta):
    a = np.dot(pu-pb, pu-pb)
    b = -2 * (2*np.dot(pu, pu) + np.dot(pb, pb) - 3*np.dot(pu, pb))
    c = np.dot(2*pu-pb, 2*pu-pb) - delta**2
    d = np.sqrt(b**2 - 4*a*c)
    t1 = (-b + d) / (2*a)
    t2 = (-b - d) / (2*a)
    if 0 <= t1 <= 2:
        if 0 <= t2 <= 2:
            return min(t1, t2)
        return t1
    elif 0 <= t2 <= 2:
        return t2
    else:
        raise ArithmeticError('Tau no está en [0,2]: %d %d', t1, t2)
    
def Dog_Leg_method(x_k,Hessian,grad,delta): #se ocupa el punto, el hessiano y el gradiente para poder calcular el 
    pb=-1*np.dot(la.inv(Hessian),grad(x_k))
    if la.norm(pb)<=delta:
        return pb
    pu=-1*(np.dot(grad(x_k).T,grad(x_k))/np.dot(grad(x_k).T,np.dot(Hessian,grad(x_k))))*grad(x_k)
    if la.norm(pu) >= delta:
        return (delta / la.norm(pu)*(1-1e-3) *pu)
    taw = resolver_taw_para_dogleg(pu, pb, delta)
    if taw <= 1:
        return taw * pu
    else:
        return pu + (taw - 1)*(pb - pu)
    

def model(f, grad, b, x_k, p, delta):  #f es la funcion objetivo, grad el gradiente, b la f
    if la.norm(p) > delta + 1e-9: #El error ese es por los cálculos de tanto producto punto que acarrea demasiados errores 
        raise ArithmeticError('P no debe de ser mas grande que delta:', p, la.norm(p), delta) #si ya se pasa demasiado de la región de confianza entonces ya hay error 
    return f(x_k) + np.dot(grad(x_k).T, p) + 0.5*np.dot(np.dot(p.T,b), p)

def trust_region_Dogleg(f, g, hf, x0, delta_0, max_delta, etha, eps=1e-5):
    x = x0
    delta = delta_0
    iterations = 0
    Hessian=Modify_hessian(x,hf)
    while True:
        iterations += 1
        b = Modify_hessian(x,hf)
        p = Dog_Leg_method(x,b,g,delta)
        rho = (f(x) - f(x+p)) / (model(f, g, b, x, p, delta) - model(f, g, b, x+p, p, delta))
        if rho < .25:
            delta = .25 * delta
        elif rho >= .75 and np.isclose(la.norm(p), delta, 1e-4):
            delta = min(2*delta, max_delta)
        if rho > etha:
            x = x + p
        elif np.allclose(p, np.zeros(p.shape), eps):
            result = x + p
            break
    
    return  result, g(result),iterations
    

In [9]:
#Rosenbrock para n=2, caso de prueba para el algoritmo
def f(datos): #Función objetivo o target function
    x,y=datos
    return 100*(y-x**2)**2+(1-x)**2

def gradiente_f(datos): #gradiente de la función f
    g=np.zeros(len(datos))
    g[0]=2*(200*datos[0]**3-200*datos[0]*datos[1]+datos[0]-1)
    g[1]=200*(datos[1]-datos[0]**2)
    return g

def hessiano_f(datos):
    hes=np.zeros((len(datos),len(datos)))
    hes[0][0]=2*(600*datos[0]**2-200*datos[1]+1)
    hes[1][0]=-400*datos[0]
    hes[0][1]=-400*datos[0]
    hes[1][1]=200
    return hes


In [10]:
res,grad_res,iteraciones=trust_region_Dogleg(f, gradiente_f, hessiano_f, [-1.2,1],0.5,1, 0.2)

In [11]:
print(iteraciones, '&' ,la.norm(grad_res), '&' ,f(res))

17306 & 4.131708020883563e-06 & 1.8786632936469468e-11


In [12]:
def Model_grad(x_k,z,b,grad):
    return grad(x_k)+np.dot(b,z)

def LSTR_method(x_k,b,grad,delta,grad_model,tol_d,maxiter):
    iterations=0
    z=np.zeros_like(x_k)
    d=-Model_grad(x_k,z,b,grad)
    p_k=None
    while la.norm(d)>tol_d and iterations<maxiter:
        if np.dot(d.T,np.dot(b,d))<=0:
            p_k=z-((np.dot(grad(x_k).T,d)+np.dot(z.T,np.dot(b,d)))/np.dot(d.T,np.dot(b,d)))*d
            if la.norm(p_k)>delta:
                return p_k/la.norm(p_k)*delta
        else:
            alfa=-(np.dot(grad(x_k).T,d)+np.dot(z.T,np.dot(b,d)))/np.dot(d.T,np.dot(b,d))
            z=z+alfa*d
            if la.norm(z)>=delta:
                p_k=z-((np.dot(grad(x_k).T,d)+np.dot(z.T,np.dot(b,d)))/np.dot(d.T,np.dot(b,d)))*d
                if la.norm(p_k)>delta:
                    return p_k/la.norm(p_k)*delta
            d=-Model_grad(x_k,z,b,grad)
            iterations=iterations+1
            
    return p_k if p_k!=None else z
  

In [13]:
def trust_region_LSTR(f, g, hf, x0, delta_0, max_delta, etha,grad_model, eps=1e-5,tol_d=1e-4,maxiter=2000):
    x = x0
    delta = delta_0
    iterations = 0
    Hessian=Modify_hessian(x,hf)
    while True:
        iterations += 1
        b = Modify_hessian(x,hf)
        p = LSTR_method(x,b,g,delta,grad_model,tol_d,maxiter)
        rho = (f(x) - f(x+p)) / (model(f, g, b, x, p, delta) - model(f, g, b, x+p, p, delta)+1e-9)
        if rho < .25:
            delta = .25 * delta
        elif rho >= .75 and np.isclose(la.norm(p), delta, 1e-4):
            delta = min(2*delta, max_delta)
        if rho > etha:
            x = x + p
        elif np.allclose(p, np.zeros(p.shape), eps):
            result = x + p
            break
    return  result, g(result),iterations
    

In [14]:
res,grad_res,iteraciones=trust_region_LSTR(f, gradiente_f, hessiano_f, [-1.2,1],0.2,1, 0.2,Model_grad)

In [15]:
print(res,la.norm(grad_res),iteraciones)

[0.99989605 0.99979169] 9.299744119917776e-05 81


## Probar las funciones Rosenbruck para n=100

In [16]:
#Definamos la función que vamos a usar 
def rosen(x): #La función de rosembruck en general
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def r_der(x):
        xm = x[1:-1]
        xm_m1 = x[:-2]
        xm_p1 = x[2:]
        der =np.zeros(x.shape)
        der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
        der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
        der[-1] = 200*(x[-1]-x[-2]**2)
        return der

def rosen_hess(x):
        H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
        diagonal = np.zeros(len(x))
        diagonal[0] = 1200*x[0]-400*x[1]+2
        diagonal[-1] = 200
        diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
        H = H + np.diag(diagonal)
        return H

In [17]:
p_inicial=np.ones(100)
p_inicial[1]=-1.2
p_inicial[98]=-1.2

res,grad_res,iteraciones=trust_region_Dogleg(rosen, r_der, rosen_hess, p_inicial,0.5,1, 0.2)

In [18]:
print("Dogleg")

print(iteraciones, '&' ,la.norm(grad_res), '&' ,rosen(res))

Dogleg
27007 & 1.485514026182182e-05 & 1.9441371533020164e-10


In [19]:
res,grad_res,iteraciones=trust_region_LSTR(rosen, r_der, rosen_hess, p_inicial,0.5,1, 0.2,Model_grad)

In [20]:
print("LSTR")
print(iteraciones, '&' ,la.norm(grad_res), '&' ,rosen(res))

LSTR
90 & 9.989178293055755e-05 & 5.0517939853093575e-09
